### РОССИЙСКИЙ УНИВЕРСИТЕТ ДРУЖБЫ НАРОДОВ

#### Факультет физико-математических и естественных наук  
#### Кафедра информационных технологий 

## ОТЧЕТ ПО ЛАБОРАТОРНОЙ РАБОТЕ № 2


### Дисциплина: Интеллектуальный анализ данных

##### Студент:  Даутов Самир Амирович
##### Группа:   НПИбд-01-19


## Москва 2022
***

### Вариант № 29

1.	При помощи модуля sqlite3 откройте базу данных Instacart в файле instacart.db.

In [ ]:
import numpy as np
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder
import itertools
import pandas as pd
import sqlite3

conn = sqlite3.connect('instacart.db')

2.  Загрузите таблицы `departments` и `products` в датафреймы Pandas. При помощи запроса SELECT извлеките из таблицы `order_products__train` записи, соответствующие указанным в индивидуальном задании дню недели (поле `order_dow` таблицы `orders`) и коду департамента (поле `department_id` таблицы `products`) и загрузите в датафрейм Pandas. Определите количество строк в полученном датафрейме и определите количество товаров (стоолбец `product_id`) в транзакциях датафрейма.

In [ ]:
pd.read_sql_query("SELECT * FROM departments", conn)[:5]

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [ ]:
pd.read_sql_query("SELECT * FROM products", conn)[:5]

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [ ]:
data = pd.read_sql_query(
    """SELECT opt.order_id, opt.product_id, add_to_cart_order, reordered, product_name 
    FROM order_products__train as opt, orders as ord, products as pr WHERE ord.order_id = opt.order_id AND pr.product_id = opt.product_id 
    AND ord.order_dow=1 AND pr.department_id=11""", conn)

In [ ]:
data.head()

,order_id,product_id,add_to_cart_order,reordered,product_name
0,819960,27,4,0,Complete Spring Water Foaming Antibacterial Ha...
1,1853924,75,9,0,"Hemp Protein, Organic"
2,2050865,75,3,1,"Hemp Protein, Organic"
3,2178635,75,2,0,"Hemp Protein, Organic"
4,251148,113,1,0,Infant's Blend Probiotic


Kоличество строк в полученном датафрейме

In [ ]:
len(data)

3045

количество товаров (стоолбец `product_id`) в транзакциях датафрейма.

In [ ]:
len(data['product_id'].unique())

1561

3.	Определите пять наиболее популярных товаров в датафрейме транзакций и определите количество покупок (транзакций) этих товаров.

Пять наиболее популярных товаров

In [ ]:
data['product_name'].value_counts()[:5]

Lavender Hand Soap                                            42
Cotton Swabs                                                  39
Lemon Verbena Hand Soap                                       29
Fluoride-Free Antiplaque & Whitening Peppermint Toothpaste    28
Premium Epsom Salt                                            26
Name: product_name, dtype: int64

4.	Постройте транзакционную базу данных из полученного датафрейма, используя в качестве идентификатора транзакции столбец `order_id`, а в качестве названий товаров - поле `product_name` из датафрейма для таблицы `products`, соответствующее столбцу `product_id`. Найдите в транзакционной базе данных три транзакции с наибольшим количеством товаров и выведите их на экран. 

In [ ]:
dataset = data.groupby('order_id')['product_name'].apply(list).to_dict()

dict(itertools.islice(dataset.items(), 5))

{'1000578': ['Pure Castile Peppermint Soap'],
 '1003833': ['Lemon Verbena Hand Soap'],
 '1005698': ['Natural Chocolate Flavor Whey Protein Powder'],
 '1007055': ['PM Acetaminophen Caplets'],
 '1008698': ['Extra Whitening Toothpaste']}

In [ ]:
data[['order_id', 'product_name']].groupby('order_id')['product_name'].count().sort_values(ascending=False)

order_id
3076974    10
2692189     9
1692125     9
2445511     9
2149222     9
           ..
2195503     1
2193292     1
2191145     1
2188459     1
998784      1
Name: product_name, Length: 2134, dtype: int64

In [ ]:
dataset.get('3076974')

['PM Acetaminophen Caplets',
 'Motion Sickness Relief, Less Drowsy Formula, 25 mg',
 'Green Tea & Honey Lemon Multi-Symptom Pain Reliever/Fever Reducer',
 'Cough Drop',
 'Gentle and Predictable Overnight Relief Laxative Tablets',
 'Adhesive Bandages Advanced Healing Blister Cushions',
 'Dry Touch Sunscreen SPF 30 Posted 5/21/2013 Ultra Sheer Sunblock',
 'Plus Lubricant Eye Drops',
 'Allergy Ultratab Tablets',
 'Allergy 24HR Nasal Allergy Spray - 120 Sprays']

In [ ]:
dataset.get('2692189')

['Marula Oil Treatment for all Skin Types',
 'Crystal Deodorant Mist Spray',
 'Triple Blade Cartridges',
 'Vitamin C Renewal Vitality Facial Serum',
 'Argan Oil 100% USDA Organic Moroccan',
 'Lotion, Dry Skin, Original Unscented',
 'Cell Stimulating Facial Mask',
 'Full Volume Conditioner Lavender & Biotin',
 'Acne Dote Deep Pore Wash']

In [ ]:
dataset.get('1692125')

['Turmeric Supreme Extra Strength',
 'Vitamin D3 Gummies, 1000 IU, Great Wild Berry Taste!',
 'Chocolate Energy Supplement',
 'Nordic Omega-3 Fishies 300 mg Chewables',
 'Chocolate Sleep Supplement',
 'Vegetarian Calcium + Vitamin D3 Sour Flavored Gummy Bears',
 'Sugar Free Multi-Vitamin & Mineral Gummy Bears',
 'Hand Soap Lemongrass',
 'Omega 3 Formula 1000mg']

5.	Постройте по транзакционной базе данных бинарную базу данных в формате датафрейма пакета `mlxtend`. По бинарной базе данных определите пять  наиболее популярных товаров и определите количество покупок (транзакций) этих товаров.

In [ ]:
te = TransactionEncoder()
dataset_bin = te.fit([i for i in dataset.values()]).transform([i for i in dataset.values()])
df = pd.DataFrame(dataset_bin, columns=te.columns_, index=[i for i in dataset.keys()])
df.head(5)

,1 Razor Handle and 2 Freesia Scented Razor Refills Premium BladeRazor System,"1,000 mg Vitamin C Lemon-Lime Flavored Fizzy Drink Mix - 30 PK",100% Natural Fruit Chew C Chewable Wafers Tangy Orange,100% Natural Spirulina Powder,100% Pure Australian Tea Tree Oil,100% Pure Sensitive Skin Care Grape Seed Oil,100% Whey Protein Powder Chocolate Fudge,100% Whey Protein Powder Vanilla,12 Hour Chest Congestion,12 Hour Orange Adult Cough Suppressant,...,Xtreme Defense 5 Fresh Blast Antiperspirant & Deodorant,Ylang Ylang + Cedarwood Hand Soap,Zinc Elderberry and Raspberry Lozenges,Zinc Target Mins 50 Mg Gluten Free Tablets,ZzzQuil Liquid Warming Berry Flavor Sleep-Aid,ZzzQuil Nighttime Sleep-Aid LiquiCaps,by Mennen Power Antiperspirant/Deodorant Fresh,gel hand wash sea minerals,go fresh Cool Moisture Beauty,rich kiss Olive & Aloe Moisturizer 2 in 1
1000578,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1003833,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1005698,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1007055,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1008698,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
pd.Series([(df[i]==True).sum() for i in df.columns],index=df.columns).sort_values(ascending=False)[:5]

Lavender Hand Soap                                            42
Cotton Swabs                                                  39
Lemon Verbena Hand Soap                                       29
Fluoride-Free Antiplaque & Whitening Peppermint Toothpaste    28
Premium Epsom Salt                                            26
dtype: int64

6.	При помощи указанного в индивидуальном задании метода построения популярных наборов предметов постройте популярный набор предметов с минимальной поддержкой не менее 3, имеющий максимальную длину. При отсутствии таких наборов уменьшите поддержку до 2. В случае нехватки вычислительных ресурсов (слишком долгой работы программы) при построении популярных наборов предметов сокращайте число записей в наборе данных (например, делая выборку половины записей набора).

In [ ]:
itemsets = fpgrowth(df, min_support=2/df.shape[0],use_colnames=True)
itemsets

,support,itemsets
0,0.004686,(Pure Castile Peppermint Soap)
1,0.013590,(Lemon Verbena Hand Soap)
2,0.004217,(Natural Chocolate Flavor Whey Protein Powder)
3,0.001874,(PM Acetaminophen Caplets)
4,0.005623,(Extra Whitening Toothpaste)
...,...,...
565,0.000937,"(Cotton Swabs, Oil-Free Eye Makeup Remover Cle..."
566,0.000937,"(2 in 1 Whitening Toothpaste, Liquid Hand Soap..."
567,0.000937,"(Soap, Pure-Castile, All-One Hemp Almond, All ..."
568,0.000937,"(Rosemary Hand Soap, Lavender Hand Soap)"


7.	Используя пакет `mlxtend` или реализацию на Python, постройте набор ассоциативных правил для полученного популярного наборов предметов. Используйте уровень достоверности (confidence), равный 0.6.

In [ ]:
data = association_rules(itemsets, metric="confidence", min_threshold=0.6)
data.tail()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Coconut Milk Nourishing Shampoo),(Coconut Milk Nourishing Conditioner),0.000937,0.002343,0.000937,1.000000,426.800000,0.000935,inf
1,(All-One Hemp Citrus Castile Bar Soap),(All One Hemp Lavender Castile Soap Bar),0.000937,0.005623,0.000937,1.000000,177.833333,0.000932,inf
2,"(Soap, Pure-Castile, All-One Hemp Almond)",(All One Hemp Lavender Castile Soap Bar),0.001406,0.005623,0.000937,0.666667,118.555556,0.000929,2.983130
3,(Smooth as Silk Conditioner),(Smooth as Silk Shampoo),0.001406,0.000937,0.000937,0.666667,711.333333,0.000936,2.997188
4,(Smooth as Silk Shampoo),(Smooth as Silk Conditioner),0.000937,0.001406,0.000937,1.000000,711.333333,0.000936,inf


8.	Для построенного набора ассоциативных правил вычислите показатель (меру) оценки ассоциативных правил, указанную в индивидуальном задании, и определите ассоциативные правила с наилучшим значением показателя оценки. 

In [ ]:
data.sort_values('leverage', ascending=False)[['antecedents','consequents', 'leverage']][:5]

,antecedents,consequents,leverage
3,(Smooth as Silk Conditioner),(Smooth as Silk Shampoo),0.000936
4,(Smooth as Silk Shampoo),(Smooth as Silk Conditioner),0.000936
0,(Coconut Milk Nourishing Shampoo),(Coconut Milk Nourishing Conditioner),0.000935
1,(All-One Hemp Citrus Castile Bar Soap),(All One Hemp Lavender Castile Soap Bar),0.000932
2,"(Soap, Pure-Castile, All-One Hemp Almond)",(All One Hemp Lavender Castile Soap Bar),0.000929
